# Data Manipulation

In this module, we will explore how to effectively do data manipulation in R.

If you are unfamiliar with notebooks, please review some basics [here](https://github.com/michhar/useR2016-tutorial-jupyter). 

## Essential Tips

A very brief summary of the critical components and commands within jupyter are:

1. Critically, press `Ctrl+Enter` to run (or render) the current cell.
2. Output will print to the notebook. You may have to scroll up to see it all.
3. Get help for any function by typing a question mark and then its name into
   the console: `?rxLinMod`. It will split the window, and will bring up the documentation for 
   that function below.
5. Files will appear in the specified directory. You can find them by selecting File in the menu bar and selecting "Open...". This will open a new browser window with a file navigator.
6. R objects can be viewed by typing `ls()` in an R cell.
7. Run all the example code!

There are a number of hands-on exercises in the document, so while you can run the notebook from beginning to end, you will get a lot more out of it by actually walking through cell-by-cell, and filling out the corresponding exercises.

These notebooks are based on a tutorial presented at a Microsoft conference in June of 2016. The original files are available [here](https://github.com/joseph-rickert/MLADS_JUNE_2016).

## Tidy Data

The biggest part of a data analysis project is preparing the data. This includes cleaning the data, deciding which variables to use, dealing with missing values and just wrestling the data into a form that is useful. Hadley Wickham, a statistician and prolific R package deeveloper, has addressed this process of wrangling data on conceptual level with his notion of "tidy data" and has put this concept into play with a series of packages that simplify the various processes of data manipulation. The examples in this module are based on using functions in Hadley's dplyr package. First a definition of Tidy Data from Hadley's JSS paper that you can find here:http://vita.had.co.nz/papers/tidy-data.pdf

Tidy data is a standard way of mapping the meaning of a dataset to its structure. A dataset is
messy or tidy depending on how rows, columns and tables are matched up with observations,
variables and types. 

In tidy data: 

1. each variable forms a column, 
2. each observation forms a row, and 
3. each type of observational unit forms a table.

The two most relevant packages for helping with this front are `dplyr` and `tidyr`, both of which we'll explore momentarily.

Before we get started, though, let's source a configuration file in the next cell. It simply makes sure that the relevant R packages and datasets are available. You do not need to look at it, but if you are interested, you can view the configuration file [here](Resources/config.R). As a side note, it also sets the `stringsAsFactors` option to FALSE, so when we read in character variables, they won't automatically turn to factors. 

It may take a few moments to run the first time you run it, but it should be fast afterwards.

In [ ]:
source("Resources/config.R")

## Data for this Module
In this module we will use two data sets: the IBM stock data from the module on [reading data into R](2-Reading_csv_files.ipynb) and another data set from Yahoo Finance on IBM dividend data.

Let's go ahead and fetch the data to a local file if we don't already have it.

In [ ]:
data_dir <- 'data'
fileName <- "ibm_stock.csv"
file <- file.path(data_dir, fileName)
if(file.exists(file)){
    cat("*** stock file already exists. Loading local copy.\n")
    ibm_stock <- read.csv(file, header = TRUE)
}else{
    cat("*** Reading stock file from remote URL.\n")
    url <- "http://real-chart.finance.yahoo.com/table.csv?s=IBM&d=1&e=1&f=2016&g=d&a=0&b=2&c=1962&ignore=.csv"
    ibm_stock <- read.csv(url,header=TRUE,sep=",")
    cat("*** Saving stock file locally for use later.\n")
    write.csv(ibm_stock, file = file)
}
head(ibm_stock, 3)

## Reading in the other file

Now, let's get the additional file and save it locally as well.

In [ ]:
file2 <- file.path(data_dir, 'ibm_div.csv')
if(file.exists(file2)){
   cat('*** dividend file already exists locally, reading from there!\n')
   ibm_div <- read.csv(file2, header = TRUE)
}else{
    cat("*** reading dividend file from remote URL...\n")
    url2 <- "http://real-chart.finance.yahoo.com/table.csv?s=IBM&a=00&b=2&c=1962&d=01&e=6&f=2016&g=v&ignore=.csv"
    ibm_div <- read.csv(url2,header=TRUE)
    cat("*** saving dividend data locally for use later...\n")
    write.csv(ibm_div, file = file2)
}
head(ibm_div, 3)


## Augmenting a Data Frame

The first function from dplyr that we will play with is the `mutate()` function. The `mutate()` function is very useful as it allows us to create an arbitrary number of transformations by just specifying them inline. In this case, we do two things:

- we create a measure `Volatility` that measures how much difference there was within the day as a function of its opening price
- we cast the Date variable so that it is a propert date variable, rather than just a character string.

In [ ]:
library(dplyr)
class(ibm_stock$Date)
ibm_stock <- mutate(ibm_stock, Volatility = (High - Low)/Open,
                   Date = as.Date(Date))
class(ibm_stock$Date)
head(ibm_stock)

## Subsetting rows of a Data Frame

The `filter()` function allows us to subset rows. We've previously seen the use of row indexing using square brackets, but `filter()` can be a little easier to read.

Here we create a new data frame contianing IBM stock data that is on or later than 1/1/2000. We do this by filtering out the rows with older data.  

In [ ]:
IBMge2000 <- filter(ibm_stock, Date >= as.Date('2000-01-01'))
head(IBMge2000)
tail(IBMge2000)
max(IBMge2000$Date)
min(IBMge2000$Date)

## Aggregating Data

Here we aggragate the daily stock data in our original IBM data frame into monthly data. The first step we will do is to use `mutate()` (in combination with the lubridate package) to create new variables that represent the month and year of each observation. Then we will create a grouped data frame, and then we will use the `summarise()` function to actually aggregate for each of the levels.



In [ ]:
library(lubridate)
ibm_stock <- mutate(ibm_stock,
              Month = month(Date), # Add variable Month
              Year  = year(Date))  # Add variable Year 

head(ibm_stock,2)
sapply(ibm_stock,class) # look at what type the variables are

Next, we group the data using the `group_by()` function, and then use the `summarise()` function to roll up the data.

In [ ]:
by_yr_mo <- group_by(ibm_stock,Year,Month) # denote the grouping variables
head(by_yr_mo)

## Now actually roll up and summarize

In [ ]:
## 
ibm_a <- summarise(by_yr_mo,
                       count = n(),
                       m.Open = mean(Open, na.rm = TRUE),
                       m.High = mean(High, na.rm = TRUE),
                       m.Low  = mean(Low, na.rm = TRUE),
                       m.Close = mean(Close, na.rm = TRUE),
                       m.Volume = mean(Volume, na.rm = TRUE),
                       m.Adj.Close = mean(Adj.Close, na.rm = TRUE),
                       m.Volatility = mean(Volatility, na.rm = TRUE))

head(ibm_a); 
tail(ibm_a); 
dim(ibm_a)

## Chaining Operations

Now, a lot of the power in dplyr actually comes from creating workflows that allow seamless concatenation of multiple steps. This is particularly powerful when we leverage the pipe operate from the magrittr package (which is loaded when dplyr is loaded). For instance, we can load the data, do mutations, filter, group data frames and summarize in a single workflow.


In [ ]:
read.csv(file, header = TRUE) %>%
 mutate(Volatility = (High - Low)/Open,
        Date = as.Date(Date),
        Month = month(Date),
        Year = year(Date)) %>%
 filter(Year > 1999) %>%
 group_by(Year, Month) %>%
 summarise(count = n(), m.Vol = mean(Volatility), sd.Vol = sd(Volatility)) -> stock_agg_by_yr_mo
head(stock_agg_by_yr_mo)

## Additional aggregations

If we try to aggregate on an already aggregated data frame, then we will systematically knock off the lowest levels of the grouping variables. In this case, we will aggregated over Months if we aggregate `stock_agg_by_yr_mo`


In [ ]:
summarise(stock_agg_by_yr_mo, 
          ndays_contributing = sum(count), 
          n = n(), 
          m.m.Vol = mean(m.Vol), 
          m.sd.Vol = mean(sd.Vol)
         )

## Merging Data Frames

In this section we merge the IBM stock file with the IBM dividend file and create a new data frame to hold the merged data. We do a "right join" which will keep all of the rows in the IBM_div data frame and only include rows from the IBM stock data frame with dates that match a dividend date. This new data frame includes the dividends and stock pricies on the days the dividends were paid.


In [ ]:
ibm_div <- mutate(ibm_div, Date = as.Date(Date)) # Make Date into a proper date data type
class(ibm_div$Date)
ibm2 <- right_join(ibm_stock,ibm_div,by="Date")        # Merge the data 
head(ibm2)

## Sorting

We can also use the `arrange()` function from dplyr to facilitate sorting.

In [ ]:
head(ibm2,2)
ibm2 <- arrange(ibm2, Date)                # Sort by date in ascending order
head(ibm2,2)
ibm2 <- arrange(ibm2, desc(Date))                # Sort by date in descending order
head(ibm2,2)


## Reshaping a Data Frame

One often needs to reshape a data frame from either long format to wide format or the other way around. What long and wide mean is just illustrated by example, but basically wide format is what you might see in an Excel spreadsheet and long format is generally what statisticians want: one row for every combination of data.

To illustrate these transformations we will use a subset of the aggregated IBM data.

Specifically, we will just extract the `Year`, `Month`, and average closing price (`m.Close`) from the aggregated data using the `select()` function:

In [ ]:
ibm_subset <- select(ibm_a, Year, Month, m.Close)
head(ibm_subset)

This dataset is in long format, so we will reshape it into wide format. It is in long format because each observation has a single row. We will reshape it into wide format using functions from the `tidyr` package

In [ ]:
library(tidyr)
ibm_wide <- spread(ibm_subset, key = Month, value = m.Close)
head(ibm_wide,3)

Next, we provide names for the columns. Note that R has a built in vector that gives abbreviated names for the months. R has many such convenience variables.

In [ ]:
names(ibm_wide)[2:13] <- month.abb    
head(ibm_wide,3)

Now we wll go back to long format and compare the new long format data frame with what we started out with. We will use the `gather()` function to do this. Note that the fourth argument interprets the `:` operator as the sequence of column names between `Jan` and `Dec`

In [ ]:
ibm_long <- gather(ibm_wide,key = Month,value = Close, Jan:Dec)
ibm_long <- arrange(ibm_long,Year,Month)          # Sort the data frame
head(ibm_long,3)                                  # New long format data frame
head(ibm_subset, 3)                               # What we started with

For more information on these spread() and arrange() functions see the following tutorial:
http://www.cookbook-r.com/Manipulating_data/Converting_data_between_wide_and_long_format/